In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv("training_twitter_x_y_train.csv")
df1=pd.read_csv("test_twitter_x_test.csv")

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 12 columns):
tweet_id                  10980 non-null int64
airline_sentiment         10980 non-null object
airline                   10980 non-null object
airline_sentiment_gold    31 non-null object
name                      10980 non-null object
negativereason_gold       24 non-null object
retweet_count             10980 non-null int64
text                      10980 non-null object
tweet_coord               776 non-null object
tweet_created             10980 non-null object
tweet_location            7430 non-null object
user_timezone             7403 non-null object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB
None


In [6]:
print(df.airline_sentiment.value_counts())

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64


In [7]:
df.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)
df1.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)

## DATA CLEANING

In [8]:
from  nltk.tokenize  import WordPunctTokenizer
tok = WordPunctTokenizer()

In [9]:
import re
from bs4 import BeautifulSoup

In [10]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [11]:
Training=df.text
Testing=df1.text

In [12]:
Train_result=[]
Test_result=[]
for i in Training:
    Train_result.append((cleaner(i)))
for i in Testing:
    Test_result.append((cleaner(i)))

## Testing Score

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## Vectorizer

In [128]:
Tfid_Vectorizer=TfidfVectorizer(ngram_range=(1,2),max_features=2000)
X_train_feature=Tfid_Vectorizer.fit_transform(Train_result)
X_test_feature=Tfid_Vectorizer.transform(Test_result)

In [129]:
print(X_train_feature.shape)
print(X_test_feature.shape)

(10980, 2000)
(3660, 2000)


## Logistic


In [130]:
from sklearn.linear_model import LogisticRegression
abc=LogisticRegression(max_iter=20000)
#grid = {'C': [1e2,1e3, 5e3, 1e4, 5e4, 1e5],}

In [131]:
#abc=GridSearchCV(clf,grid)

In [132]:
abc.fit(X_train_feature,df.airline_sentiment)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=20000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [133]:
Sentiment1=abc.predict(X_test_feature)

In [134]:
np.savetxt('SentimentSVM.csv',Sentiment1,fmt="%s")

In [135]:
abc.score(X_train_feature,df.airline_sentiment)

0.8297814207650274